In [1]:
import os
import hdf5storage
import numpy as np
import pandas as pd
# from sklearn.preprocessing import StandardScaler
# from sklearn.feature_selection import f_classif
# from sklearn.decomposition import PCA
# from sklearn.model_selection import train_test_split
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import accuracy_score
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dropout, GRU, LSTM, Dense
# from tensorflow.keras.optimizers import Adam
# from sklearn.metrics import confusion_matrix, roc_curve, precision_recall_curve, auc
# from sklearn.metrics import ConfusionMatrixDisplay, RocCurveDisplay, PrecisionRecallDisplay

# import seaborn as sns
# import matplotlib.pyplot as plt
#
# from sklearn.preprocessing import MinMaxScaler

import pandas as pd
from pgmpy.estimators import PC
from pgmpy.models import BayesianModel
from pgmpy.estimators import MaximumLikelihoodEstimator

from scipy.stats import entropy

import os
import pandas as pd
from scipy.io import loadmat

import networkx as nx
import matplotlib.pyplot as plt

In [2]:
"""
Load Data Seperate All data in Protective and Non-Prtective Data
"""

# Assuming the 'Data' folder is in the current working directory
data_folders = ['./Data']

# Initialize lists to store the separated dataframes
protective_dfs = []
non_protective_dfs = []

# Loop through each data folder
for data_folder in data_folders:
    # List all .mat files in the current data folder
    mat_files = [f for f in os.listdir(data_folder) if f.endswith('.mat')]
    # Load each mat file
    for mat_file in mat_files:
        # Construct the full path to the .mat file
        mat_path = os.path.join(data_folder, mat_file)
        # Load the .mat file
        mat_data = loadmat(mat_path)
        # Convert the data into a pandas dataframe
        df = pd.DataFrame(mat_data['data'])
        # Select only the first 70 columns and the last column (73rd) which contains the behavior label
        df = df.iloc[:, list(range(66)) + [70] + [72]]
        # Split the data based on the protective behavior label
        # Assuming the last column in df is the protective behavior label
        protective_behavior = df.iloc[:, -1]
        protective_df = df[protective_behavior == 1]
        non_protective_df = df[protective_behavior == 0]
        # Append the resulting dataframes to their respective lists
        protective_dfs.append(protective_df)
        non_protective_dfs.append(non_protective_df)

# Concatenate all protective and non-protective dataframes
all_protective_data = pd.concat(protective_dfs, axis=0, ignore_index=True)
all_non_protective_data = pd.concat(non_protective_dfs, axis=0, ignore_index=True)

# Now `all_protective_data` and `all_non_protective_data` hold the protective and non-protective data respectively
# You can process these dataframes as needed for your analysis or save them to new .mat files
print(all_protective_data.shape)
print(all_non_protective_data.shape)

(77298, 68)
(437247, 68)


In [3]:
nodes_data = {}

# There are 22 nodes, so we loop through each
for node in range(1, 23):
    # Calculate the index for x, y, and z based on the node number
    x_index = node - 1
    y_index = x_index + 22
    z_index = x_index + 44

    # Extract the data for the current node
    node_data = all_non_protective_data.iloc[:, [x_index, y_index, z_index]]

    # Assign the node data to the corresponding entry in the dictionary
    nodes_data[f'node_{node}'] = node_data

In [4]:
 # (node_from, node_to) based on the known graph structure
edges = [
    (1, 2), (1, 5), (1, 8), (2, 3), (3, 4), (5, 6), (6, 7),
    (8, 9), (9, 10), (9, 15), (9, 20), (10, 11), (11, 12),
    (12, 13), (13, 14), (15, 16), (16, 17), (17, 18), (18, 19),
    (9, 20), (20, 21), (21, 22)
]

In [5]:
def calculate_kl_divergence(p, q, epsilon=1e-10):
    p_normed = (p + epsilon) / (np.sum(p) + epsilon * len(p))
    q_normed = (q + epsilon) / (np.sum(q) + epsilon * len(q))
    return entropy(p_normed, q_normed)

def analyze_joint_interactions(node_data_1, node_data_2, labels=('node_1', 'node_2')):
    """Analyzes interactions between two nodes and calculates weight based on the direction of influence."""
    directions = []
    kl_divergences = {f'{labels[0]}_to_{labels[1]}': [], f'{labels[1]}_to_{labels[0]}': []}

    for i in range(3):  # For X, Y, Z coordinates
        df = pd.DataFrame({
            labels[0]: node_data_1[i].iloc[:30000],
            labels[1]: node_data_2[i].iloc[:30000]
        })

        pc = PC(df)
        estimated_dag = pc.estimate()
        bn = BayesianModel(estimated_dag.edges())
        bn.fit(df, estimator=MaximumLikelihoodEstimator)

        cpd_for_node_1 = bn.get_cpds(node=labels[0])
        cpd_for_node_2 = bn.get_cpds(node=labels[1])

        p_node_1_given_2 = cpd_for_node_1.values
        p_node_2_given_1 = cpd_for_node_2.values

        kl_divergence_1_to_2 = np.mean(calculate_kl_divergence(p_node_1_given_2, p_node_2_given_1))
        kl_divergence_2_to_1 = np.mean(calculate_kl_divergence(p_node_2_given_1, p_node_1_given_2))

        direction_key = f'{labels[0]}_to_{labels[1]}' if kl_divergence_1_to_2 < kl_divergence_2_to_1 else f'{labels[1]}_to_{labels[0]}'
        directions.append(direction_key)
        kl_divergences[direction_key].append(min(kl_divergence_1_to_2, kl_divergence_2_to_1))

    final_direction = max(set(directions), key=directions.count)
    final_weight = np.mean(kl_divergences[final_direction])

    return final_direction, final_weight

In [6]:
"""
Extracting X, Y, Z coordinates for node from the protective data,
"""
node_1_data = [nodes_data['node_1'][0], nodes_data['node_1'][22], nodes_data['node_1'][44]]
node_2_data = [nodes_data['node_2'][1], nodes_data['node_2'][23], nodes_data['node_2'][45]]
node_3_data = [nodes_data['node_3'][2], nodes_data['node_3'][24], nodes_data['node_3'][46]]
node_4_data = [nodes_data['node_4'][3], nodes_data['node_4'][25], nodes_data['node_4'][47]]
node_5_data = [nodes_data['node_5'][4], nodes_data['node_5'][26], nodes_data['node_5'][48]]
node_6_data = [nodes_data['node_6'][5], nodes_data['node_6'][27], nodes_data['node_6'][49]]
node_7_data = [nodes_data['node_7'][6], nodes_data['node_7'][28], nodes_data['node_7'][50]]
node_8_data = [nodes_data['node_8'][7], nodes_data['node_8'][29], nodes_data['node_8'][51]]
node_9_data = [nodes_data['node_9'][8], nodes_data['node_9'][30], nodes_data['node_9'][52]]
node_10_data = [nodes_data['node_10'][9], nodes_data['node_10'][31], nodes_data['node_10'][53]]
node_11_data = [nodes_data['node_11'][10], nodes_data['node_11'][32], nodes_data['node_11'][54]]
node_12_data = [nodes_data['node_12'][11], nodes_data['node_12'][33], nodes_data['node_12'][55]]
node_13_data = [nodes_data['node_13'][12], nodes_data['node_13'][34], nodes_data['node_13'][56]]
node_14_data = [nodes_data['node_14'][13], nodes_data['node_14'][35], nodes_data['node_14'][57]]
node_15_data = [nodes_data['node_15'][14], nodes_data['node_15'][36], nodes_data['node_15'][58]]
node_16_data = [nodes_data['node_16'][15], nodes_data['node_16'][37], nodes_data['node_16'][59]]
node_17_data = [nodes_data['node_17'][16], nodes_data['node_17'][38], nodes_data['node_17'][60]]
node_18_data = [nodes_data['node_18'][17], nodes_data['node_18'][39], nodes_data['node_18'][61]]
node_19_data = [nodes_data['node_19'][18], nodes_data['node_19'][40], nodes_data['node_19'][62]]
node_20_data = [nodes_data['node_20'][19], nodes_data['node_20'][41], nodes_data['node_20'][63]]
node_21_data = [nodes_data['node_21'][20], nodes_data['node_21'][42], nodes_data['node_21'][64]]
node_22_data = [nodes_data['node_22'][21], nodes_data['node_22'][43], nodes_data['node_22'][65]]

In [7]:
final_direction, final_weight = analyze_joint_interactions(node_1_data, node_2_data, ('node_1', 'node_2'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_1_to_node_2, Final weight: 10.109714902473439


In [8]:
final_direction, final_weight = analyze_joint_interactions(node_1_data, node_5_data, ('node_1', 'node_5'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_1_to_node_5, Final weight: 10.109714902473439


In [9]:
final_direction, final_weight = analyze_joint_interactions(node_1_data, node_8_data, ('node_1', 'node_8'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_1_to_node_8, Final weight: 10.109714902473439


In [10]:
final_direction, final_weight = analyze_joint_interactions(node_2_data, node_3_data, ('node_2', 'node_3'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_3_to_node_2, Final weight: 10.151334635707808


In [11]:
final_direction, final_weight = analyze_joint_interactions(node_3_data, node_4_data, ('node_3', 'node_4'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_4_to_node_3, Final weight: 10.156386161971218


In [12]:
final_direction, final_weight = analyze_joint_interactions(node_5_data, node_6_data, ('node_5', 'node_6'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_5_to_node_6, Final weight: 10.147894470070922


In [13]:
final_direction, final_weight = analyze_joint_interactions(node_6_data, node_7_data, ('node_6', 'node_7'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_6_to_node_7, Final weight: 10.15642417842419


In [14]:
final_direction, final_weight = analyze_joint_interactions(node_8_data, node_9_data, ('node_8', 'node_9'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_8_to_node_9, Final weight: 10.147894470070607


In [15]:
final_direction, final_weight = analyze_joint_interactions(node_9_data, node_10_data, ('node_9', 'node_10'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_9_to_node_10, Final weight: 10.147894470081196


In [16]:
final_direction, final_weight = analyze_joint_interactions(node_9_data, node_15_data, ('node_9', 'node_15'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_15_to_node_9, Final weight: 10.170716297505477


In [17]:
final_direction, final_weight = analyze_joint_interactions(node_9_data, node_20_data, ('node_9', 'node_20'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_20_to_node_9, Final weight: 10.170716297505479


In [18]:
final_direction, final_weight = analyze_joint_interactions(node_10_data, node_11_data, ('node_10', 'node_11'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_11_to_node_10, Final weight: 10.201290922888242


In [19]:
final_direction, final_weight = analyze_joint_interactions(node_11_data, node_12_data, ('node_11', 'node_12'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_11_to_node_12, Final weight: 10.210974081461668


In [20]:
final_direction, final_weight = analyze_joint_interactions(node_12_data, node_13_data, ('node_12', 'node_13'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_12_to_node_13, Final weight: 10.235100128404808


In [21]:
final_direction, final_weight = analyze_joint_interactions(node_13_data, node_14_data, ('node_13', 'node_14'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_13_to_node_14, Final weight: 10.256814577674033


In [22]:
final_direction, final_weight = analyze_joint_interactions(node_15_data, node_16_data, ('node_15', 'node_16'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_15_to_node_16, Final weight: 10.17876313155881


In [23]:
final_direction, final_weight = analyze_joint_interactions(node_16_data, node_17_data, ('node_16', 'node_17'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_16_to_node_17, Final weight: 10.20206636645345


In [24]:
final_direction, final_weight = analyze_joint_interactions(node_17_data, node_18_data, ('node_17', 'node_18'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_18_to_node_17, Final weight: 10.249176473627353


In [7]:
final_direction, final_weight = analyze_joint_interactions(node_18_data, node_19_data, ('node_18', 'node_19'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_18_to_node_19, Final weight: 10.257736872249723


In [8]:
final_direction, final_weight = analyze_joint_interactions(node_20_data, node_21_data, ('node_20', 'node_21'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_20_to_node_21, Final weight: 10.17876313155667


In [9]:
final_direction, final_weight = analyze_joint_interactions(node_21_data, node_22_data, ('node_21', 'node_22'))
print(f"Final direction: {final_direction}, Final weight: {final_weight}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Final direction: node_21_to_node_22, Final weight: 10.238546714376003
